<a href="https://colab.research.google.com/github/theidari/alphabet_soup/blob/main/src/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="#880808"><h1><b>Alphabet Soup Charity Pre-Optimization Attempt</b></h1></font>


In [1]:
# Delete the existing directory
!rm -rf alphabet_soup

# Clone the repository to a new directory
!git clone https://github.com/theidari/alphabet_soup.git

# Dependencies and setup
from alphabet_soup.src.package.constants import * # constants
from alphabet_soup.src.package.helpers import * # liberaries and functions

Cloning into 'alphabet_soup'...
remote: Enumerating objects: 340, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 340 (delta 74), reused 8 (delta 1), pack-reused 227
Receiving objects: 100% (340/340), 551.25 KiB | 2.19 MiB/s, done.
Resolving deltas: 100% (227/227), done.
☑ constants is imporetd
☑ helpers is imporetd


In [2]:
# Loading the data into a Pandas DataFrame
application_df = pd.read_csv(DATA_URL)
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


<font color="#880808"><h3><b>Step 1: Removing unnecessary columns</b></h3><font>

In [3]:
# drop 'EIN', 'NAME', 'SPECIAL_CONSIDERATIONS' columns
drop_application_df=application_df.drop(['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS'], axis=1)

# drop 'IS_SUCCESSFUL' by correlation condition 
if drop_application_df['ASK_AMT'].corr(drop_application_df['IS_SUCCESSFUL']) < 0.1:
    drop_application_df=drop_application_df.drop(['ASK_AMT'], axis=1)

# drop 'IS_SUCCESSFUL' by correlation condition
if len(drop_application_df['STATUS'].unique()) == 2 and drop_application_df['STATUS'].corr(drop_application_df['IS_SUCCESSFUL']) < 0.1:
    drop_application_df=drop_application_df.drop(['STATUS'], axis=1)

# display df   
drop_application_df    

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1
...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,0
34297,T5,Independent,C3000,ProductDev,Association,0,1


In [4]:
# cutoff APPLICATION_TYPE value and create a list of application types to be replaced
binning (drop_application_df,'APPLICATION_TYPE',500)


--------------------------------------------------------------------------------
 Value Count before binning:
--------------------------------------------------------------------------------
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
--------------------------------------------------------------------------------
Value Count after binning:
--------------------------------------------------------------------------------
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# cutoff CLASSIFICATION value and create a list of application types to be replaced
binning (drop_application_df,'CLASSIFICATION',800)


--------------------------------------------------------------------------------
 Value Count before binning:
--------------------------------------------------------------------------------
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
--------------------------------------------------------------------------------
Value Count after binning:
--------------------------------------------------------------------------------
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(drop_application_df)

In [7]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(['IS_SUCCESSFUL'], axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

input_features=len(X_train_scaled[1])

<font color="#880808"><h3><b>Step 2: Determine the optimal number of layers using the "wider and deeper" approach, and to identify the optimal number of epochs using the "callback stop" method.</b></h3><font>

In [8]:
# test wider approach
epoch_func([160, 60], "relu", 256, 500, [input_features, X_train_scaled, y_train, X_test_scaled, y_test])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 160)               6400      
                                                                 
 dense_1 (Dense)             (None, 60)                9660      
                                                                 
 dense_2 (Dense)             (None, 1)                 61        
                                                                 
Total params: 16,121
Trainable params: 16,121
Non-trainable params: 0
_________________________________________________________________
None


Minimum validation loss: 0.5542421340942383
268/268 - 0s - loss: 0.5542 - accuracy: 0.7249 - 327ms/epoch - 1ms/step
Loss: 0.5542418956756592, Accuracy: 0.7248979806900024


In [9]:
# test wider + deeper approach
epoch_func([320, 320, 120, 120], "relu", 256, 500, [input_features, X_train_scaled, y_train, X_test_scaled, y_test])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 320)               12800     
                                                                 
 dense_4 (Dense)             (None, 320)               102720    
                                                                 
 dense_5 (Dense)             (None, 120)               38520     
                                                                 
 dense_6 (Dense)             (None, 120)               14520     
                                                                 
 dense_7 (Dense)             (None, 1)                 121       
                                                                 
Total params: 168,681
Trainable params: 168,681
Non-trainable params: 0
_________________________________________________________________
None


Minimum validation loss: 0.5534011721611023
268/268 - 0s - loss: 0.5534 - accuracy: 0.7265 - 494ms/epoch - 2ms/step
Loss: 0.5534013509750366, Accuracy: 0.7265306115150452


In [10]:
# test deeper approach
epoch_func([320, 320, 320, 120, 120, 120], "relu", 1000, 500, [input_features, X_train_scaled, y_train, X_test_scaled, y_test])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 320)               12800     
                                                                 
 dense_9 (Dense)             (None, 320)               102720    
                                                                 
 dense_10 (Dense)            (None, 320)               102720    
                                                                 
 dense_11 (Dense)            (None, 120)               38520     
                                                                 
 dense_12 (Dense)            (None, 120)               14520     
                                                                 
 dense_13 (Dense)            (None, 120)               14520     
                                                                 
 dense_14 (Dense)            (None, 1)                

Minimum validation loss: 0.5539891719818115
268/268 - 1s - loss: 0.5540 - accuracy: 0.7272 - 543ms/epoch - 2ms/step
Loss: 0.5539892315864563, Accuracy: 0.7272303104400635


In [11]:
# create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh',"sigmoid"])
    
    # allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=320,
        step=5), activation=activation, input_dim=input_features))

    # allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 8)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # compile the model
    nn_model.compile(loss="mean_squared_error", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [12]:
# import the kerastuner library
!pip install -q -U keras-tuner
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=35,
    hyperband_iterations=2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 5.2 MB/s eta 0:00:00


In [13]:
# run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=35,validation_data=(X_test_scaled,y_test)) 

Trial 180 Complete [00h 01m 46s]
val_accuracy: 0.7266472578048706

Best val_accuracy So Far: 0.7294460535049438
Total elapsed time: 01h 19m 53s


In [14]:
# get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 301, 'num_layers': 5, 'units_0': 81, 'units_1': 36, 'units_2': 41, 'units_3': 11, 'units_4': 116, 'units_5': 76, 'units_6': 81, 'units_7': 46, 'tuner/epochs': 35, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 71, 'num_layers': 8, 'units_0': 96, 'units_1': 21, 'units_2': 51, 'units_3': 96, 'units_4': 6, 'units_5': 16, 'units_6': 41, 'tuner/epochs': 35, 'tuner/initial_epoch': 12, 'tuner/bracket': 2, 'tuner/round': 2, 'units_7': 1, 'tuner/trial_id': '0070'}
{'activation': 'relu', 'first_units': 211, 'num_layers': 3, 'units_0': 116, 'units_1': 26, 'units_2': 56, 'units_3': 111, 'units_4': 91, 'units_5': 41, 'units_6': 1, 'units_7': 26, 'tuner/epochs': 35, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [15]:
# evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.1871 - accuracy: 0.7294 - 782ms/epoch - 3ms/step
Loss: 0.18714749813079834, Accuracy: 0.7294460535049438
268/268 - 1s - loss: 0.1888 - accuracy: 0.7292 - 1s/epoch - 5ms/step
Loss: 0.18878240883350372, Accuracy: 0.7292128205299377
268/268 - 2s - loss: 0.1867 - accuracy: 0.7290 - 2s/epoch - 7ms/step
Loss: 0.18670891225337982, Accuracy: 0.7289795875549316


In [16]:
# first best model 
best_model_1 = top_model[0]
best_model_1.build()
best_model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 301)               12040     
                                                                 
 dense_1 (Dense)             (None, 81)                24462     
                                                                 
 dense_2 (Dense)             (None, 36)                2952      
                                                                 
 dense_3 (Dense)             (None, 41)                1517      
                                                                 
 dense_4 (Dense)             (None, 11)                462       
                                                                 
 dense_5 (Dense)             (None, 116)               1392      
                                                                 
 dense_6 (Dense)             (None, 1)                 1

In [17]:
# second best model 
best_model_2 = top_model[1]
best_model_2.build()
best_model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 71)                2840      
                                                                 
 dense_1 (Dense)             (None, 96)                6912      
                                                                 
 dense_2 (Dense)             (None, 21)                2037      
                                                                 
 dense_3 (Dense)             (None, 51)                1122      
                                                                 
 dense_4 (Dense)             (None, 96)                4992      
                                                                 
 dense_5 (Dense)             (None, 6)                 582       
                                                                 
 dense_6 (Dense)             (None, 16)                1

In [18]:
# third best model 
best_model_3 = top_model[2]
best_model_3.build()
best_model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 211)               8440      
                                                                 
 dense_1 (Dense)             (None, 116)               24592     
                                                                 
 dense_2 (Dense)             (None, 26)                3042      
                                                                 
 dense_3 (Dense)             (None, 56)                1512      
                                                                 
 dense_4 (Dense)             (None, 1)                 57        
                                                                 
Total params: 37,643
Trainable params: 37,643
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Export our model to HDF5 file
best_model_1.save(OUTPUT_URL+"AlphabetSoupCharity_PreOptimization.h5")